# Tringulate points from knwon camera poses with Pycolmap


In [3]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import pycolmap
from deep_image_matching.triangulation import (
    create_db_from_model,
    import_keypoints,
    import_matches,
    import_verifed_matches,
)
from deep_image_matching.utils import (
    OutputCapture,
)

# Define project directory
root_path = Path("/home/francesco/phd/deep-image-matching/datasets/belv_20230725")

# Path to the images
image_dir = root_path / "images"

# Path to the pre-computed COLMAP reconstuction with the knwon poses
sfm_path = root_path / "results_superpoint+lightglue_bruteforce_quality_high"
sfm_db_path = sfm_path / "database.db"
sfm_rec_path = sfm_path / "reconstruction"

# Path to the dense matching results to be triangulated
dense_path = root_path / "results_roma_bruteforce_quality_high.1"
features_h5 = dense_path / "features.h5"
matches_h5 = dense_path / "matches.h5"
pair_file = dense_path / "pairs.txt"

# Path to the output for the dense matching
dense_db_path = dense_path / "database_dense.db"
model_path = dense_path / "dense_model"
model_path.mkdir(exist_ok=True, parents=True)

# Do geometric verification of the dense features (with known camera poses)
do_geometric_verification = True
max_error = 4

In [5]:
# Import the sparse reconstruction
sfm_rec = pycolmap.Reconstruction(sfm_rec_path)

# Create an empty database from the sparse reconstruction
image_ids = create_db_from_model(sfm_rec, dense_db_path)

# Add keypoints to the database
import_keypoints(features_h5, image_ids, dense_db_path)

# Add matches to the database, but do not add two-view geometry
import_matches(
    matches_h5,
    image_ids,
    dense_db_path,
    pair_file,
    add_two_view_geometry=not do_geometric_verification,
)


if do_geometric_verification:
    # Run the geometric verification with the knwon camera poses and add the inliers matches to the database in the two-view geometry table
    import_verifed_matches(
        image_ids,
        sfm_rec,
        dense_db_path,
        features_h5,
        matches_h5,
        pair_file,
        max_error=max_error,
    )

2024-04-08 15:25:32 | [WARNING ] The database already exists, deleting it.


Importing matches: 100%|██████████| 1/1 [00:00<00:00, 352.17it/s]

2024-04-08 15:25:33 | [INFO    ] Performing geometric verification of the matches...



Importing verified matches: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s]

2024-04-08 15:25:33 | [INFO    ] mean/med/min/max valid matches 70.36/70.36/70.36/70.36%.


In [6]:
# Run the triangulation with the known camera poses

# Define the options for the triangulation according to the IncrementalPipelineOptions available in pycolmap
# print(pycolmap.IncrementalPipelineOptions().summary())
opt = dict(triangulation=dict(ignore_two_view_tracks=False))
verbose = True

with OutputCapture(verbose):
    with pycolmap.ostream():
        reconstruction = pycolmap.triangulate_points(
            sfm_rec,
            dense_db_path,
            image_dir,
            model_path,
            options=opt,
        )

I20240408 15:25:37.644026 2077708 misc.cc:198] 
Loading database
I20240408 15:25:37.645126 2077708 database_cache.cc:54] Loading cameras...
I20240408 15:25:37.645164 2077708 database_cache.cc:64]  2 in 0.000s
I20240408 15:25:37.645174 2077708 database_cache.cc:72] Loading matches...
I20240408 15:25:37.646092 2077708 database_cache.cc:78]  1 in 0.001s
I20240408 15:25:37.646107 2077708 database_cache.cc:94] Loading images...
I20240408 15:25:37.679128 2077708 database_cache.cc:143]  2 in 0.033s (connected 2)
I20240408 15:25:37.679203 2077708 database_cache.cc:154] Building correspondence graph...
I20240408 15:25:37.706593 2077708 database_cache.cc:190]  in 0.027s (ignored 0)
I20240408 15:25:37.707015 2077708 timer.cc:91] Elapsed time: 0.001 [minutes]
I20240408 15:25:37.717986 2077708 misc.cc:198] 
Triangulating image #1 (0)
I20240408 15:25:37.718021 2077708 sfm.cc:473] => Image sees 0 / 131622 points
I20240408 15:25:38.029822 2077708 sfm.cc:478] => Triangulated 131620 points
I20240408 15:

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  6.183404e+04    0.00e+00    1.44e+03   0.00e+00   0.00e+00  1.00e+04        0    8.18e-02    3.27e-01
   1  5.939892e+04    2.44e+03    1.43e-01   2.36e-01   1.00e+00  3.00e+04        0    1.15e-01    4.43e-01


I20240408 15:25:38.691607 2077708 misc.cc:205] 
Bundle adjustment report
------------------------
I20240408 15:25:38.691670 2077708 bundle_adjustment.cc:942] 
    Residuals : 526480
   Parameters : 394860
   Iterations : 2
         Time : 0.46784 [s]
 Initial cost : 0.342707 [px]
   Final cost : 0.335891 [px]
  Termination : Convergence

I20240408 15:25:38.706387 2077708 incremental_mapper.cc:175] => Completed observations: 0
I20240408 15:25:38.717056 2077708 incremental_mapper.cc:178] => Merged observations: 0
I20240408 15:25:38.741540 2077708 incremental_mapper.cc:160] => Filtered observations: 0
I20240408 15:25:38.741565 2077708 sfm.cc:521] => Changed observations: 0.000000
I20240408 15:25:38.834072 2077708 misc.cc:198] 
Extracting colors


In [7]:
# Export the model in ply and bundler format
reconstruction.export_PLY(model_path / "dense_model.ply")
reconstruction.export_bundler(model_path / "bundler.out", model_path / "bunlder_list.txt", skip_distortion=True)